In [2]:
import os

In [3]:
%pwd

'/Users/aayushaggarwal/Desktop/Chicken-disease-project/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/Users/aayushaggarwal/Desktop/Chicken-disease-project'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_log_root_dir: Path
    checkpoint_model_file_path: Path

In [23]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml,create_directories
import tensorflow as tf

In [24]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def prepare_callbacks_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_chkpoint_dir = os.path.dirname(config.checkpoint_model_file_path)
        create_directories([Path(model_chkpoint_dir),
                            Path(config.tensorboard_log_root_dir)
                            ])
        prepare_callbacks_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_root_dir= Path(config.tensorboard_log_root_dir),
            checkpoint_model_file_path= Path(config.checkpoint_model_file_path)

        )
        return prepare_callbacks_config

    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken_disease")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_path=Path(prepare_base_model.updated_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [25]:
import time


In [26]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_root_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_file_path),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [27]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [28]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [29]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-03-24 16:55:13,417: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-24 16:55:13,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-24 16:55:13,419: INFO: common: created directory at: artifacts]
[2024-03-24 16:55:13,421: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_model_dir]
[2024-03-24 16:55:13,421: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log]
[2024-03-24 16:55:13,422: INFO: common: created directory at: artifacts/training]
Found 402 images belonging to 2 classes.
Found 1612 images belonging to 2 classes.
Epoch 1/10
50/50 [==============================] - 186s 4s/step - loss: 9.9823 - accuracy: 0.5608 - val_loss: 4.0504 - val_accuracy: 0.6354
Epoch 2/10


/Users/aayushaggarwal/anaconda3/envs/chicken/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - 351s 7s/step - loss: 1.3307 - accuracy: 0.8696 - val_loss: 0.9425 - val_accuracy: 0.8880
Epoch 3/10
50/50 [==============================] - 217s 4s/step - loss: 1.1147 - accuracy: 0.8861 - val_loss: 0.6849 - val_accuracy: 0.9245
Epoch 4/10
50/50 [==============================] - 243s 5s/step - loss: 0.5539 - accuracy: 0.9177 - val_loss: 1.4473 - val_accuracy: 0.8542
Epoch 5/10
50/50 [==============================] - 257s 5s/step - loss: 0.5170 - accuracy: 0.9241 - val_loss: 0.8049 - val_accuracy: 0.9062
Epoch 6/10
50/50 [==============================] - 260s 5s/step - loss: 1.2757 - accuracy: 0.8886 - val_loss: 0.5035 - val_accuracy: 0.9375
Epoch 7/10
50/50 [==============================] - 262s 5s/step - loss: 0.2914 - accuracy: 0.9424 - val_loss: 0.4749 - val_accuracy: 0.9453
Epoch 8/10
50/50 [==============================] - 262s 5s/step - loss: 0.2245 - accuracy: 0.9544 - val_loss: 0.4592 - val_accuracy: 0.9427
Epoch 9/10
50/50 [======